In [60]:
# Choose what you want to use among various kinds of features
using_1type = 1
using_2type = 1
using_3type = 1
using_4type = 1

##############################################################################################
import numpy as np
from matplotlib import pyplot 
import pandas as pd
from pandas import read_csv 
from pandas import set_option 
from pandas.tools.plotting import scatter_matrix 
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import classification_report 
from sklearn.metrics import precision_recall_fscore_support 
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.linear_model import LogisticRegression 
from sklearn.linear_model import PassiveAggressiveClassifier 
from sklearn.linear_model import SGDClassifier 
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.naive_bayes import GaussianNB 
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier 
from sklearn.ensemble import GradientBoostingClassifier 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier
from data_handler import *
from sklearn.ensemble import VotingClassifier
from sklearn.neural_network import MLPClassifier

feature_path = 'assets/feature-vec/'
tempdata_path = 'assets/temp-data/'

### Data Load
FE_X_train = load(feature_path+'FE_X_train')
FE_X_test = load(feature_path+'FE_X_test')
################################### FE
FE2_X_train = load(feature_path+'FE2_X_train')
FE2_X_test = load(feature_path+'FE2_X_test')
################################### FE 2
FE3_X_train = load(feature_path+'FE3_X_train')
FE3_X_test = load(feature_path+'FE3_X_test')
################################### FE 3
FE4_X_train = load(feature_path+'FE4_X_train')
FE4_X_test = load(feature_path+'FE4_X_test')
#Y_train = load('pre_Y_train')
#Y_test = load('pre_Y_test')
Y_train = load(tempdata_path+'Ytrain')
Y_test = load(tempdata_path+'Ytest')
################################### Y
feature_info_without_LK = load('feature_info_without_LK')

### Choice Feature Set 
X_train, X_test = choice_feature_set(using_1type, using_2type, using_3type, using_4type,\
                                     FE_X_train, FE2_X_train, FE3_X_train, FE4_X_train,\
                                     FE_X_test, FE2_X_test, FE3_X_test, FE4_X_test)
print('===> Total feature length =', len(X_train[0][0]))

### SHOW FEATURE INFORMATION
# for row in feature_info_without_LK:
#     print(row)




################################
""" Remove unnecessary words """
################################
#pre_X_train = load('pre_X_train') # preprocessed text data
#pre_X_test = load('pre_X_test') # preprocessed text data

# delete_list = ['the', ',', 'a', 'to']
# X_train, Y_train = del_ele_in_li(delete_list, X_train, Y_train, for_X=True, one_dim=False)
# X_test, Y_test = del_ele_in_li(delete_list, X_test, Y_test, for_X=True, one_dim=False)





###################################################
""" Unfold Data (from sequence to non-sequence) """
###################################################
unfolded_X_train, unfolded_Y_train = unfold_data(X_train, Y_train)
unfolded_X_test, unfolded_Y_test = unfold_data(X_test, Y_test)


assert(len(X_train) == len(Y_train))
assert(len(unfolded_X_train) == len(unfolded_Y_train))
assert(len(unfolded_X_train[0]) == len(unfolded_X_test[0]))
assert(len(X_test) == len(Y_test))

===> Total feature length = 10121


## Univariate Selection

In [61]:
# # Feature Extraction with Univariate Statistical Tests (Chi-squared for classification) 
# from pandas import read_csv 
# from numpy import set_printoptions 
# from sklearn.feature_selection import SelectKBest 
# from sklearn.feature_selection import chi2 



# test = SelectKBest(score_func=chi2, k=1000) 
# fit = test.fit(unfolded_X_train, unfolded_Y_train) 

# X_train_uni = fit.transform(unfolded_X_train)
# X_test_uni = fit.transform(unfolded_X_test) 

## PCA

In [62]:
# from sklearn.decomposition import PCA

# pca = PCA(n_components=1000) 
# fit = pca.fit(unfolded_X_train)

# X_train_pca = pca.transform(unfolded_X_train)
# X_test_pca = pca.transform(unfolded_X_test)

## Tuning Hyperparameter

In [63]:
# # Grid Search for Algorithm Tuning 
# from sklearn.model_selection import GridSearchCV 
# n_estimators = np.array([10, 50, 100, 300]) 
# param_grid = dict(n_estimators=n_estimators) 
# model = RandomForestClassifier() 
# grid = GridSearchCV(estimator=model, param_grid=param_grid) # default 3-fold cross validation
# grid.fit(unfolded_X_train, unfolded_Y_train) 
# print(grid.best_score_) 
# print(grid.best_estimator_.n_estimators)

In [64]:
np.array(unfolded_X_test).shape

(2553, 10121)

## Scikit-learn ML Models

* LR 중 LASSO LR이 더 효과있으리라 생각이 된다. 우리의 feature는 0이 매우 많은 sparse한 feature이기 때문이다.
* LR 중 stepwise LR도 있다.
* data set이 적을 때, cross-validation으로 모델 검증을 해도 된다. (development set없이) 하지만, 그럴려면, 코딩이 더 필요한데, 사전을 오로지 training data만 보고 구축해야 되기 때문이다. 따라서, scikit-learn에서 제공해주는 cross-validation함수를 사용하면 안된다.

In [65]:
# Test options and evaluation metric 
# num_folds = 10
# seed = 7 
# scoring = 'accuracy'

models = []
""" Linear Models """
#models.append(('Logistic Regression', LogisticRegression(penalty='l2', C=0.4)))
#models.append(('Logistic Regression', LogisticRegression(penalty='l1', C=0.4)))
# models.append(('Logistic Regression', LogisticRegression(solver='liblinear')))
# models.append(('Logistic Regression', LogisticRegression(solver='newton-cg')))
# models.append(('Logistic Regression', LogisticRegression(solver='lbfgs')))
# models.append(('Logistic Regression', LogisticRegression(solver='sag')))
#models.append(('Logistic Regression', LogisticRegression(solver='liblinear',penalty='l2', tol=0.00001, max_iter=100, verbose=1)))
# models.append(('LR 1', LogisticRegression(C=10000.0, penalty='l2')))
# models.append(('LR 2', LogisticRegression(C=0.9)))
# models.append(('LR 2', LogisticRegression(C=0.8)))
# models.append(('LR 2', LogisticRegression(C=0.7)))
# models.append(('LR 2', LogisticRegression(C=0.6)))
# models.append(('LR 3', LogisticRegression(C=0.5)))
# models.append(('LR 3', LogisticRegression(C=0.4)))
# models.append(('LR 3', LogisticRegression(C=0.3)))
# models.append(('LR 3', LogisticRegression(C=0.2)))
# models.append(('LR 4', LogisticRegression(C=10000.0, penalty='l1')))
# models.append(('LR 5', LogisticRegression(C=100.0, penalty='l1')))
# models.append(('LR 6', LogisticRegression(C=0.01, penalty='l1')))

#models.append(('PassiveAggressiveClassifier', PassiveAggressiveClassifier(C=2000)))

#models.append(('SGDClassifier', SGDClassifier(alpha = 0.00001))) # alpha = 0.0001 : default
# 모델이 underfit이라는 것을 확인하였다. 위와 같이 l1, l2 penalty를 사용하니까 모두 성능이 떨어졌다.

#models.append(('LDA', LinearDiscriminantAnalysis())) # 진짜 오래걸림
# models.append(('LDA', LinearDiscriminantAnalysis(tol=0.0001))) # default
#models.append(('LDA', LinearDiscriminantAnalysis(tol=0.1)))
#models.append(('LDA', LinearDiscriminantAnalysis(tol=0.9))) # best?
#models.append(('LDA', LinearDiscriminantAnalysis(tol=1.0)))
#models.append(('LDA', LinearDiscriminantAnalysis(tol=10)))

""" Ensemble Models """
#models.append(('ET', ExtraTreesClassifier(n_estimators=800))) # best
#models.append(('ET', ExtraTreesClassifier(n_estimators=100))) 
#models.append(('RF', RandomForestClassifier(n_estimators=1500))) 
#models.append(('AB', AdaBoostClassifier(n_estimators=3000))) # default: 50
#models.append(('GBM', GradientBoostingClassifier())) # 진짜 오래걸림

""" Non-linear Models """
#models.append(('CART', DecisionTreeClassifier()))
#models.append(('KNN', KNeighborsClassifier(n_neighbors = 7)))
#models.append(('NB', GaussianNB()))
#models.append(('LinearSVM', LinearSVC()))
#models.append(('SVM', SVC()))
models.append(('SVM', SVC(C=300)))
models.append(('SVM', SVC(C=200)))
models.append(('SVM', SVC(C=100)))
models.append(('SVM', SVC(C=50)))
models.append(('SVM', SVC(C=10)))
""" Voting Ensemble """
# create the sub models 
estimators = [] 
estimators.append(('logistic', LogisticRegression(penalty='l2', C=0.4) )) 
estimators.append(('et', ExtraTreesClassifier(n_estimators=800) )) 
estimators.append(('svm', SVC(C=900)))
#estimators.append(('rf', RandomForestClassifier() )) 
#estimators.append(('cart', DecisionTreeClassifier() )) 
# create the ensemble model 
#models.append(('Voting', VotingClassifier(estimators)))  

""" Neural Network """
#models.append(('MLP', MLPClassifier())) # 


###################
""" Train Model """
###################
results = []
names = []
for name, model in models:
#     kfold = KFold(n_splits=num_folds, random_state=seed)
#     cv_results = cross_val_score(model, unfolded_X_train, unfolded_Y_train, cv=kfold, scoring=scoring)
#     results.append(cv_results)
#     names.append(name)
#     msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
#     print(msg)

    # just fit
    model.fit(unfolded_X_train, unfolded_Y_train)
    #model.fit(X_train_pca, unfolded_Y_train)
    
# ### COMPARE MODELS USING PLOT
# fig = pyplot.figure() 
# fig.suptitle('Model Comparison') 
# ax = fig.add_subplot(111) 
# pyplot.boxplot(results) 
# ax.set_xticklabels(names) 
# pyplot.show()

########################
""" Evaluation Model """
########################
# Make predictions on validation dataset 
# knn = KNeighborsClassifier() 
# knn.fit(X_train, Y_train) 
# predictions = knn.predict(X_validation) 

for name, model in models:
    print('======================= <', name, '> =======================')
    predictions = model.predict(unfolded_X_test)
    #predictions = model.predict(X_test_pca)
    print(accuracy_score(unfolded_Y_test, predictions))
    print(confusion_matrix(unfolded_Y_test, predictions))
    print(classification_report(unfolded_Y_test, predictions))
    
    pre, rec, f1, sup = (precision_recall_fscore_support(unfolded_Y_test, predictions, average='micro'))
    print('micro => ', pre, '\t', rec, '\t', f1, '\t', sup)
    pre, rec, f1, sup = (precision_recall_fscore_support(unfolded_Y_test, predictions, average='macro'))
    print('macro => ', pre, '\t', rec, '\t', f1, '\t', sup)
    pre, rec, f1, sup = (precision_recall_fscore_support(unfolded_Y_test, predictions, average='weighted'))
    print('weighted => ',pre, '\t', rec, '\t', f1, '\t', sup)
    print('\n')
    report = classification_report(unfolded_Y_test, predictions)
    
#print(using_1type, using_2type, using_3type, using_4type)

======================= < SVM > =======================
0.768115942029
[[ 75   0   0   0   1   0   0   0  12   0   0   0   0   0   0   0   4]
 [  3   6   1   6   2   0   0   0   2   4   0   0   0   0   0   0   6]
 [  1   0 145   3   0   2   1   0   7   0   0   0   0   0   0   0   4]
 [  2   0   6 215   6   0   3   0  17  61   1   0   0   0   0   0  26]
 [  1   0   2  28 103   0   2   0   6   8   0   5   0   0   0   0   7]
 [  0   0   7   0   0 197   0   0   1   0   1   0   0   0   0   0   5]
 [  0   0   1   8   4   1 114   0   0   4   3   0   0   1   1   0   7]
 [  2   0   0   0   0   1   0 139   0   0   0   0   0   0   0   0   0]
 [  8   0   1  24   1   0   3   0 217  12   1   0   0   0   5   0   9]
 [  0   2   1  56   1   0   6   0   8 158   2   1   1   0   1   0  12]
 [  0   0   4   1   0   0   0   0   0   2  11   0   0   0   0   0   0]
 [  0   0   0   7   0   0   0   0   1  12   0  32   0   0   0   0   1]
 [  0   0   0   0   0   0   1   0   0   0   0   0   7   0   0   0   0]
 [  0 

C:\Users\Office\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.650607128868
[[ 71   0   0   1   0   0   0   1  13   1   0   0   0   0   0   0   5]
 [  2   0   0   7   6   0   0   0   2   4   0   0   0   0   0   0   9]
 [  1   0 131   4   0   8   1   0  11   1   0   0   0   0   0   0   6]
 [  0   0   8 205   4   1   2   0  17  61   0   0   0   0   0   0  39]
 [  1   0   4  42  64   0   5   0   6  15   0   0   0   0   0   0  25]
 [  2   0   6   0   0 186   0   0   4   0   0   0   0   0   0   0  13]
 [  0   0   6  14   1   2  96   0   3   2   0   0   0   0   0   0  20]
 [ 28   0   0   0   0   0   0 108   1   0   0   0   0   0   0   0   5]
 [  5   0   5  23   0   0   1   0 200  26   0   0   0   0   0   0  21]
 [  2   0   5 100   0   1   3   0  10  96   0   0   0   0   0   0  32]
 [  0   0  16   0   0   0   0   0   0   0   0   0   0   0   0   0   2]
 [  0   0   0  44   0   0   0   0   1   7   0   0   0   0   0   0   1]
 [  0   0   0   0   1   1   6   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   1   0  10   0   0   0   0   0   0   0   0 